## Solving rooming and inbasket scheduling problem

At clinics, MA have two large responsibilities. 1st is to room patients and assist doctor with any administrative and logistical support for patients the physician is seeing. Physician needs the MA to bring patients from waiting room, take their vitals, take chief complaint, do any paperwork not done by front desk, prepare any materials for procedures or injections. The other responsibility is inbasket management or inbasket. This responsibility entails managing the messages the physicians receive in their inbasket which may range from patient messages, pharmancy requests, laboratory notifications etc. Many messages can be done with the MA authority, others need physician approval, physician input, or completely managed by physician. 

Generally, every physician who is seeing patient needs an MA to room for that physician(ie asssigned to room). Also, every physician in the group needs to have an MA covering for physician inbasket (ie assigned to inbasket) 

### Algorithm for rooming
This schedule needs to be created first because:
1. Its a more important schedule than inbasket. Direct patient care provides value to the clinic because they can bill for it. 
2. Physician get upset when they do not have a MA to help them with tasks. They get frustrated when they are asked to do tasks which the MA usually does for a number of reasons. 

#### Inputs: 
1. **Physician schedule:** indicating which ones are working, and need an MA scheduled by AM, PM 
2. **MA schedule:** indicating which ones are available to room by AM and PM
3. **MA_physician pairs:** indicates which physicians to ma pairings
4. **Weights:** indicates how much work it is to work with a physician. Perhaps their schedule is changed. 

#### Outputs 
1. **Physician to ma pairs** dictionary that maps physician name to list of tuples. Each tuple will be in form of (time, ma_name), where time is in this case AM or PM. na_name is 
2. **Ma to physician pairs**: dictionary that maps ma name to list of typles. Each tuple 


In [7]:
# What input should look like

physician_schedule = {"Dr.Liz": ["AM", "PM"],
                         "Dr.Wong": ["AM", "PM"],
                         "Dr.Res": ["AM"]}

ma_schedule = {"Sally":["AM", "PM"], 
                "Diego": ["AM", "PM"], 
                "Lucero": ["AM"]}

physician_schedule = [{"key":"Dr.Liz","AM":True,"PM":True}, 
                      {"key":"Dr.Wong","AM":True,"PM":True},
                      {"key":"Dr.Res","AM":True,"PM":False},
                      {"key":"Dr. Gonzalez","AM":False,"PM":False},
                     {"key":"Dr. Su","AM":False,"PM":False},
                     {"key":"Dr. Nelligan","AM":False,"PM":False}]

ma_schedule =  [{"key":"Sally","AM":True,"PM":True}, 
                      {"key":"Diego","AM":True,"PM":True},
                      {"key":"Lucero","AM":True,"PM":False},
                      {"key":"Alex","AM":True,"PM":True},
               ]
rooming = {"Sally":[("AM", "Dr.Wong"),("PM", "Dr.Wong")], 
                         "Diego": [("AM", "Dr.Res"),("PM", "Dr.Liz")], 
                         "Lucero": [("AM", "Dr.Liz")]}

inbasket =         {"Sally":[("AM", "Dr.Wong"),("PM", "Dr.Wong")], 
                         "Diego": [("AM", "Dr.Res"),("PM", "Dr.Liz")], 
                         "Lucero": [("AM", "Dr.Liz")],
                        "Alex":[("AM", "Dr.Gonzales"), ("AM", "Nelligan"),("PM", "Dr.Liz"),("PM", "Dr.Gonzales"), ("PM", "Nelligan")]}

preferences = [{"key":"Dr.Wong",
                "preference": ["Sally"]}]

# What output should look like 

# physcian to MA
# [{key" doctor", AM_MA: "sally", PM_MA: "diego"}, ]

physician_to_ma_output = {"Dr.Liz": [("AM", "Lucero"),("PM", "Diego")],
                         "Dr.Wong": [("AM", "Sally"),("PM", "Sally")], 
                         "Dr.Res": [("AM", "Dr.Res")]}

weights = {"Dr.Liz": 1, "Dr.Wong": 1, "Dr.Res": 1}



#### Setting up as Integer programming solution

Solving:

1. Apply preferences
2. Ofthose without preferences, apply integer problem. optimizing for equal work. 
3. 

In [206]:
from ortools.linear_solver import pywraplp
import numpy as np

def find_rooming_inbasket_assign(physician_schedule, ma_schedule, preferences):
    
    #create variable matrix 
        #num cols will correspond to the people
        #num rows will correspond to time 
        #each cell will have an array of variables
    t_physician_schedule = {}
    for dic in physician_schedule: 
        name = dic["key"]
        value = []
        if dic["AM"]:
            value = ["AM"]
        if dic["PM"]:
            value = value + ["PM"]
        t_physician_schedule[name] = value
    physician_schedule = t_physician_schedule
    print("physician schedule is \n {}".format(physician_schedule))
    
    t_ma_schedule = {}
    for dic in ma_schedule: 
        name = dic["key"]
        print("!! {}".format(name))
        value = []
        if dic["AM"]:
            value = ["AM"]
        if dic["PM"]:
            value = value + ["PM"]
        t_ma_schedule[name] = value
    ma_schedule = t_ma_schedule
    for ma in ma_schedule.keys():
        print(ma)
    ma_names = list(ma_schedule.keys())
    print("ma names is \n {}".format(ma_names))
    
    t_preferences = {}
    for dic in preferences: 
        name = dic["key"]
        value = dic["preference"]
        
        t_preferences[name] = value
    preferences = t_preferences
    
    
    time_num_dim = 2
    time_dims = np.array(["AM", "PM"])
    matrix_vars = np.zeros((len(ma_schedule), time_num_dim), dtype=object)
    for ma in ma_schedule.keys():
        print(ma)
    ma_names = list(ma_schedule.keys())
    print("ma names is {}".format(ma_names))
    doctor_names = list(physician_schedule.keys())
    #we will create a matrix with size (num ma, time periods)
    #each cell will contain a list where it has all relevant variables
    for row in range(matrix_vars.shape[0]):
        for col in range(matrix_vars.shape[1]):
            matrix_vars[row, col] = []
    
    solver = pywraplp.Solver('SolveIntegerProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    #add rooming variables 
    for doctor, schedule in physician_schedule.items():  
        for time_period in schedule:
            print(time_period)
            print(np.nonzero(time_dims == time_period)[0][0])
            #TODO: will crash if time_period is not valid
            curr_col = np.nonzero(time_dims == time_period)[0][0]
            
            for row in range(matrix_vars.shape[0]):
                if time_period in ma_schedule[ma_names[row]]:
                    var_name = "rooming "+ ma_names[row] +" "+ time_period +" "+ doctor
                    print(var_name)
                    matrix_vars[row, curr_col] = matrix_vars[row, curr_col] + [solver.IntVar(0.0, 1.0, var_name)]
                else:
                    print("we got {} ma and day {}".format(ma_names[row], time_period))
    
    #add inbasket variables 
    for row in range(matrix_vars.shape[0]):
        for col in range(matrix_vars.shape[1]):
            for doctor in doctor_names:
                if time_dims[col] in ma_schedule[ma_names[row]]:
                    var_name = "inbasket "+ ma_names[row] +" "+ time_dims[col] +" "+ doctor
                    matrix_vars[row, col] = matrix_vars[row, col] + [solver.IntVar(0.0, 1.0, var_name)]
                else:
                    print("We rejected {} ma and time {}".format(ma_names[row],time_dims[col]))
    
    #set preferences 
        #let lower bound for combination of physician-time-ma for both vars equal to 1
    for doctor, pref in preferences.items():
        for row in range(matrix_vars.shape[0]):
            for col in range(matrix_vars.shape[1]):
                for var in matrix_vars[row, col]:
                    split_name = var.name().split()
                    if split_name[-1] == doctor and split_name[1] == pref[0]:
                        print("we got " + str(var.lb()))
                        print(var.name())
                        var.SetLb(1)
                        print("we got " +  str(var.lb()))
    
    # set rooming constraints 
    rooming_constraints = []
    for doctor in doctor_names:
        for col in range(matrix_vars.shape[1]):
            if time_dims[col] not in physician_schedule[doctor]:
                print("SKIPPPED")
                continue 
            rooming_constraint = solver.Constraint(1, 1)
            for row in range(matrix_vars.shape[0]):
                for var in matrix_vars[row, col]: 
                    if var.name().split()[-1] == doctor and var.name().split()[0] == "rooming":
                        print("var name is {}".format(var.name()))
                        rooming_constraint.SetCoefficient(var, 1)
            rooming_constraints = rooming_constraints + [rooming_constraint]
    print(len(rooming_constraints))
    
    #set inbasket constraints We are not making constraints right
    inbasket_constraints = []
    for doctor in doctor_names:
        for col in range(matrix_vars.shape[1]):
            inbasket_constraint = solver.Constraint(1, 1)
            print("time is {}".format(time_dims[col]))
            for row in range(matrix_vars.shape[0]):
                for var in matrix_vars[row, col]: 
                    if var.name().split()[-1] == doctor and var.name().split()[0] == "inbasket":
                        print("\t" + var.name())
                        inbasket_constraint.SetCoefficient(var, 1)
            inbasket_constraints = inbasket_constraints + [inbasket_constraint]
                        
    #set work constraints 
    z = solver.IntVar(0.0, solver.infinity(),"z")
    work_constraints = []
    for row in range(matrix_vars.shape[0]):
        for col in range(matrix_vars.shape[1]):
            constraint = solver.Constraint(-solver.infinity(), 0)
            constraint.SetCoefficient(z, -1)
            for var in matrix_vars[row, col]:
                if var.name().split()[0] == "rooming":
                    constraint.SetCoefficient(var, 3)
                elif var.name().split()[0] == "inbasket":
                    constraint.SetCoefficient(var, 1)
            work_constraints = work_constraints + [constraint] 
    
    objective = solver.Objective()
    objective.SetCoefficient(z, 1)
    objective.SetMinimization()
    result_status = solver.Solve()
    print(result_status)
    #variable used for objective ie want to min Z
    
    for row in range(matrix_vars.shape[0]):
        for col in range(matrix_vars.shape[1]):
            for var in matrix_vars[row, col]:
                print('%s = %d' % (var.name(), var.solution_value()))
    
    rooming_schedule = {}
    inbasket_schedule = {}
    
    for row in range(matrix_vars.shape[0]):
        for col in range(matrix_vars.shape[1]):
            for var in matrix_vars[row, col]:
                if var.name().split()[0] == "rooming" and var.solution_value() == 1:
                    if var.name().split()[1] not in rooming_schedule.keys():
                        rooming_schedule[var.name().split()[1]] = {var.name().split()[-2]: var.name().split()[-1]}
                    else: 
                        rooming_schedule[var.name().split()[1]][var.name().split()[-2]] = var.name().split()[-1]
                elif var.name().split()[0] == "inbasket" and var.solution_value() == 1:
                    if var.name().split()[1] not in inbasket_schedule.keys():
                        inbasket_schedule[var.name().split()[1]] = {var.name().split()[-2]: var.name().split()[-1]}
                    else: 
                        print(var.name())
                        inbasket_schedule[var.name().split()[1]][var.name().split()[-2]] = var.name().split()[-1]
                    
    return rooming_schedule, inbasket_schedule